# Part 1: Load Data

In [1127]:
import numpy as np
import pandas as pd
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
# import gensim

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Apple\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Apple\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Apple\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Apple\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Apple\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [1128]:
# Load data into dataframe
#This csv document is web-scripted by myself
hulkman = pd.read_csv(r"C:\Users\Apple\Desktop\Amazon Hulkman Jumpstarter 4-dimension.csv", error_bad_lines=False)

In [1129]:
hulkman.head()

,Stars,Title,Date,Review Body
0,5.0,Very powerful booster that is well made.,2020/12/14,Jumped off a 4.7 Durango that has been sitting...
1,1.0,Will update when number provided correct.,2020/12/14,Support phone number will not accept incoming ...
2,5.0,Hulkman is by far the BEST Jump Starter Unit o...,2020/12/19,So many uses but by far the Best Automotive J...
3,5.0,Great Roll-Out,2020/12/28,A-lot of thought went into this product right...
4,4.0,Can't get a hold of the manufacturer,2021/2/1,The charger worked great for the first couple ...


In [1130]:
hulkman.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2135 entries, 0 to 2134
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Stars        2072 non-null   float64
 1   Title        2072 non-null   object 
 2   Date         2127 non-null   object 
 3   Review Body  2132 non-null   object 
dtypes: float64(1), object(3)
memory usage: 66.8+ KB


In [1131]:
# Remove missing value
#hulkman.dropna(subset=['Stars'],inplace=True)

hulkman=hulkman.dropna(axis=0, how='any')
#https://www.yisu.com/zixun/234986.html#:~:text=python%E5%88%A0%E9%99%A4%E5%88%97%E4%B8%BA%E7%A9%BA,%E5%88%97%E4%B8%BA%E7%A9%BA%E7%9A%84%E8%A1%8C%E3%80%82

In [1132]:
hulkman.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2071 entries, 0 to 2134
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Stars        2071 non-null   float64
 1   Title        2071 non-null   object 
 2   Date         2071 non-null   object 
 3   Review Body  2071 non-null   object 
dtypes: float64(1), object(3)
memory usage: 80.9+ KB


In [1133]:
#We believe the reviews with starts>=4 are good review, while the reviews with starts<=3 are bad review.

In [1134]:
hulkman_Good=hulkman.loc[hulkman["Stars"]>=4]
hulkman_Bad=hulkman.loc[hulkman["Stars"]<=3]

In [1135]:
hulkman_Bad

,Stars,Title,Date,Review Body
1,1.0,Will update when number provided correct.,2020/12/14,Support phone number will not accept incoming ...
8,3.0,Dead Battery Jump Start Experience,2021/3/7,I bought this product a few weeks ago not expe...
9,1.0,Does not work very bad quality,2021/3/22,Stay away only worked 3 times and gave up
15,1.0,Very weak,2021/5/11,I sell cars and i use jumpers very frequently....
16,1.0,This one did not charge. After two hours of c...,2021/5/12,I watched many reviews about this Hulkman jump...
...,...,...,...,...
1898,2.0,Very poor User Guide,2021/6/22,Very poor User Guide. No information whatsoev...
1907,3.0,"GREAT booster, BUT looses it's power when cold.",2022/1/19,I purchased this item a few months ago and I h...
1962,1.0,Will not connect to battery cables.,2021/12/11,Will not connect to battery cables. I’m unabl...
1983,1.0,Useless,2021/8/9,Useless tried to jump my car battery and didn'...


In [1136]:
title=hulkman["Title"].tolist()
Good_title=hulkman_Good["Title"].tolist()
Bad_title=hulkman_Bad["Title"].tolist()

In [1137]:
Review=hulkman["Review Body"].tolist()
Good_Review=hulkman_Good["Review Body"].tolist()
Bad_Review=hulkman_Bad["Review Body"].tolist()

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library. Stop words are words like "a", "the", or "in" which don't convey significant meaning. 
Stemming is the process of breaking a word down into its root.

In [1139]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') #stopwords.append("n't")
new_stops=["starter","jump","hulkman","alpha85","jumper","battery","car","unit","thing","unit"]
for i in new_stops:
    stopwords.append(i)
    
#stopwords.append("starter")
#stopwords.append("jump")
#stopwords.append("hulkman")
#stopwords.append("alpha85")
#stopwords.append("jumper")
#stopwords.append("battery")
#stopwords.append("car")
#stopwords.append("unit")
#stopwords.append("thing")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 189 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [1140]:
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string
    #将所有stopwords(无意义)的词语删除
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    #将所有数字删除
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [1141]:
tokenization_and_stemming(Review[0])[0:15]

['jump',
 'durango',
 'sit',
 'six',
 'month',
 'zero',
 'volt',
 'twice',
 'struggl',
 'honest',
 'sound',
 'like',
 'crank',
 'new',
 'booster']

## tokenization and lemmatization

In [1143]:
# tokenization and lemmatization
#More details:https://blog.csdn.net/TFATS/article/details/109093464

# 获取单词的词性
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def tokenization_and_lemmatization(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string
    #将所有stopwords(无意义)的词语删除
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    #将所有数字删除
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
    
    tags = pos_tag(filtered_tokens) # 获取单词词性
    wnl = WordNetLemmatizer()
    lemmatization = []
    
    #lemmatization
    for tag in tags:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmatization.append(wnl.lemmatize(tag[0], pos=wordnet_pos)) # 词形还原
    
    return lemmatization

In [1144]:
tokenization_and_lemmatization(Review[0])[0:15]

['jump',
 'durango',
 'sit',
 'six',
 'month',
 'zero',
 'volt',
 'twice',
 'struggle',
 'honestly',
 'sound',
 'like',
 'crank',
 'new',
 'booster']

In [1145]:
Review[0]

'Jumped off a 4.7 Durango that has been sitting for six months with a battery at zero volts twice with no struggle at all. Honestly it sounded like it was cranking from a new battery and a booster. Better yet it only showed a ten percent drop in power battery power. The clamps are sturdy and the cables while short are of heavier gauge wire than allot of others. This is a good thing since that means less resistance. The included charger is well constructed as well. Bother the charger and the USB c cable that come with it have Velcro ties seem to them to make storage neat and easy. The worklight while not insanely bright is just about a perfect amount of light to illuminate a wide area for work while not blinding you. All in all this is one impressive jump starter that will see allot of use at work and at home. In an Auto tech and Jane used a great number of boosters from smaller units like this to massive dual deep cycle units and this will be my go to every time.'

# Part 3: Term Frequency-Inverse Document Frequency

## Analyzing the Good Review

In [1147]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram

#tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
#                                 min_df=0.01, stop_words='english',
#                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(2,3))

tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_lemmatization, ngram_range=(2,3))

tfidf_matrix = tfidf_model.fit_transform(Good_Review) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

C:\Users\Apple\AppData\Roaming\Python\Python38\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make', 'u'] not in stop_words.
  warnings.warn(


In total, there are 1892 reviews and 64 terms.


In [1148]:
tfidf_matrix

<1892x64 sparse matrix of type '<class 'numpy.float64'>'
	with 2612 stored elements in Compressed Sparse Row format>

In [1149]:
#tfidf_matrix.toarray() #todense()

In [1150]:
#tfidf_matrix.todense()

In [1151]:
#Save the terms identified by TF-IDF.

In [1152]:
# words
tf_selected_words = tfidf_model.get_feature_names()

C:\Users\Apple\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [1153]:
# print out words
tf_selected_words[0:15]

['able start',
 'carry case',
 'charge charge',
 'charge fast',
 'charge minute',
 'charge phone',
 'charge quickly',
 'charge start',
 'charge use',
 'come case',
 'come charge',
 'completely dead',
 'dead battery',
 'dead start',
 'definitely recommend']

In [1154]:
#Part 4: K-means clustering

In [1155]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [1156]:
#4.1. Analyze K-means Result

In [1157]:
# create DataFrame films from all of the input files.
product = { "Stars":hulkman_Good["Stars"],'review': hulkman_Good["Review Body"], 'cluster': clusters}
frame = pd.DataFrame(product, columns = ["Stars",'review', 'cluster'])

In [1158]:
frame.head(10)

,Stars,review,cluster
0,5.0,Jumped off a 4.7 Durango that has been sitting...,0
2,5.0,So many uses but by far the Best Automotive J...,0
3,5.0,A-lot of thought went into this product right...,1
4,4.0,The charger worked great for the first couple ...,1
5,4.0,This is a well designed emergency jump starter...,2
6,5.0,"About an hour ago, I received my Hulkman Alpha...",0
7,5.0,"The days of looking for jumper cables, then an...",0
10,5.0,I had a dinky old Napa Blue jump starter lying...,0
11,5.0,Update 5/21 Hulkman has gone above and beyond ...,1
12,4.0,"This thing charges very fast, has usb outlet f...",0


In [1159]:
frame.groupby("cluster").agg("mean")

,Stars
cluster,
0,4.937500
1,4.939394
2,4.906542
3,4.948718
4,4.842105


In [1160]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
0,1472
3,195
2,107
1,99
4,19


In [1161]:
km.cluster_centers_

# 239数的list -> cluster 0的中心点的tf-idf值
#-> assumption: 中心点的值可以代表这个cluster
#-> tf-idf值越大，对应的词越能代表这个document
#-> 选出了tf-idf最大的6个值对应的词来代表这个cluster

array([[1.07493878e-02, 1.90161623e-02, 8.49424852e-03, 1.58353312e-02,
        1.10906020e-02, 2.10809595e-02, 1.70512215e-02, 1.17627224e-02,
        1.02208958e-02, 1.03108323e-02, 9.07760535e-03, 1.04937144e-03,
        6.75538921e-03, 7.52455918e-03, 1.07197036e-02, 8.32918853e-03,
        2.06317756e-02, 5.56768600e-03, 1.33590233e-02, 4.92395128e-03,
        1.42184516e-02, 3.32269096e-02, 2.15090402e-04, 1.27329971e-02,
        3.10969049e-02, 7.14690466e-03, 1.96387899e-02, 9.84319173e-03,
        1.14383425e-02, 6.76379781e-03, 8.90222806e-03, 6.04903599e-03,
        7.33517887e-03, 1.09992043e-02, 1.14126032e-02, 1.17977922e-02,
        6.36576569e-03, 8.00809819e-03, 1.45109089e-02, 4.00610847e-04,
        0.00000000e+00, 1.36078267e-02, 9.85529735e-03, 2.42455473e-02,
        9.97849291e-03, 1.24887369e-02, 1.19534897e-02, 7.60802059e-03,
        8.04443464e-03, 7.47540730e-03, 1.10047251e-02, 8.65077401e-03,
        7.88535674e-03, 1.90226872e-02, 1.95398594e-02, 3.114794

In [1162]:
km.cluster_centers_.shape

(5, 64)

In [1163]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :15]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print()
  
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    #print (", ".join(cluster_reviews))
    print()

<Document clustering result by K-means>
Cluster 0 words:great product,use time,highly recommend,start right,charge phone,easy use,hold charge,use start,use charge,carry case,charge quickly,work like,charge fast,simple use,fully charge,
Cluster 0 reviews (1472 reviews): 

Cluster 1 words:work great,easy use,highly recommend,use work,easy use work,start right,charge fast,product work,use start,right box,hold charge,time charge,force start,great product,use time,
Cluster 1 reviews (99 reviews): 

Cluster 2 words:completely dead,start completely dead,start completely,dead start,force start,easy use,highly recommend,start right,use time,work perfectly,charge minute,start vehicle,hold button,super easy,fully charge,
Cluster 2 reviews (107 reviews): 

Cluster 3 words:easy use,use start,highly recommend,product easy,peace mind,carry case,charge phone,use twice,fast charge,start vehicle,use charge,great product,long time,simple use,come charge,
Cluster 3 reviews (195 reviews): 

Cluster 4 words

Conclusion:

Cluster 0: Fast start car and charge phone

Cluster 1: Work smoothly

Cluster 2: Able to jump start the car with completely dead battery

Cluster 3: Easy to use

Cluster 4: Heavy duty and good quality

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [1165]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [1166]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1892, 5)
[[0.59999578 0.10000118 0.10000108 0.10000091 0.10000105]
 [0.2        0.2        0.2        0.2        0.2       ]
 [0.22530517 0.07539512 0.33840908 0.07434437 0.28654626]
 ...
 [0.59999872 0.10000011 0.1000001  0.10000009 0.10000098]
 [0.2        0.2        0.2        0.2        0.2       ]
 [0.2        0.2        0.2        0.2        0.2       ]]


In [1167]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 64)
[[2.00891031e-01 2.00501620e-01 1.46845375e+01 1.59927532e+01
  2.00696631e-01 2.00874766e-01 2.00489855e-01 2.00959376e-01
  2.01595839e-01 2.00448245e-01 4.65074733e+00 2.01566323e-01
  4.05438696e-01 2.00658308e-01 2.00303818e-01 2.00992616e-01
  2.07783961e+02 1.23213774e+01 2.00905653e-01 2.00835155e-01
  2.00914084e-01 3.19905232e-01 2.00491122e-01 2.00450514e-01
  2.01219886e-01 1.37067219e+01 2.00236027e-01 2.00231745e-01
  2.00887575e-01 2.01255387e-01 2.00565768e-01 1.43256782e+01
  2.01390312e-01 2.00861393e-01 2.00762517e-01 2.03586255e+01
  2.01663068e-01 2.01890103e-01 2.00466323e-01 2.00275961e-01
  2.00258716e-01 2.33744180e+01 2.00751723e-01 3.69032570e+00
  2.00800100e-01 2.01041900e-01 2.00600830e-01 2.01002216e-01
  2.01230863e-01 2.00743816e-01 2.00364526e-01 1.51632049e+01
  2.01382357e-01 2.00941189e-01 2.01111862e-01 2.01664835e-01
  2.00988130e-01 1.96696448e+01 2.00218457e-01 2.00289111e-01
  2.01429929e-01 2.00311288e-01 2.00231745e-01 2.00228359e-01]

In [1168]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(hulkman_Good))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.60,0.10,0.10,0.10,0.10,0
Doc1,0.20,0.20,0.20,0.20,0.20,0
Doc2,0.23,0.08,0.34,0.07,0.29,2
Doc3,0.10,0.10,0.10,0.10,0.60,4
Doc4,0.07,0.07,0.73,0.07,0.07,2
Doc5,0.24,0.07,0.22,0.40,0.07,3
Doc6,0.08,0.08,0.66,0.08,0.09,2
Doc7,0.07,0.07,0.71,0.07,0.07,2
Doc8,0.10,0.10,0.10,0.10,0.60,4
Doc9,0.09,0.08,0.66,0.09,0.08,2


In [1169]:
df_document_topic['topic'].value_counts().to_frame()

,topic
0,956
3,265
4,236
2,224
1,211


In [1170]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.round(2).head()

[[2.00891031e-01 2.00501620e-01 1.46845375e+01 1.59927532e+01
  2.00696631e-01 2.00874766e-01 2.00489855e-01 2.00959376e-01
  2.01595839e-01 2.00448245e-01 4.65074733e+00 2.01566323e-01
  4.05438696e-01 2.00658308e-01 2.00303818e-01 2.00992616e-01
  2.07783961e+02 1.23213774e+01 2.00905653e-01 2.00835155e-01
  2.00914084e-01 3.19905232e-01 2.00491122e-01 2.00450514e-01
  2.01219886e-01 1.37067219e+01 2.00236027e-01 2.00231745e-01
  2.00887575e-01 2.01255387e-01 2.00565768e-01 1.43256782e+01
  2.01390312e-01 2.00861393e-01 2.00762517e-01 2.03586255e+01
  2.01663068e-01 2.01890103e-01 2.00466323e-01 2.00275961e-01
  2.00258716e-01 2.33744180e+01 2.00751723e-01 3.69032570e+00
  2.00800100e-01 2.01041900e-01 2.00600830e-01 2.01002216e-01
  2.01230863e-01 2.00743816e-01 2.00364526e-01 1.51632049e+01
  2.01382357e-01 2.00941189e-01 2.01111862e-01 2.01664835e-01
  2.00988130e-01 1.96696448e+01 2.00218457e-01 2.00289111e-01
  2.01429929e-01 2.00311288e-01 2.00231745e-01 2.00228359e-01]
 [2.012

C:\Users\Apple\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able start,carry case,charge charge,charge fast,charge minute,charge phone,charge quickly,charge start,charge use,come case,...,use start,use time,use twice,use work,wall charger,work advertise,work great,work like,work like charm,work perfectly
Topic0,0.20,0.20,14.68,15.99,0.20,0.20,0.20,0.20,0.20,0.20,...,0.20,0.20,0.20,19.67,0.20,0.20,0.20,0.20,0.2,0.2
Topic1,0.20,0.20,0.20,0.20,5.66,0.20,0.20,0.20,0.20,16.27,...,0.20,7.42,0.40,0.20,0.20,16.46,0.20,0.20,0.2,0.2
Topic2,17.07,35.19,0.20,4.01,16.25,34.23,0.20,0.20,17.57,0.20,...,0.20,10.48,0.20,0.20,12.47,0.20,0.20,0.20,0.2,23.7
Topic3,0.20,0.20,0.20,8.57,0.40,0.20,26.28,20.37,0.20,0.20,...,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.2,0.2
Topic4,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,...,40.07,34.92,18.45,0.20,0.20,0.20,79.88,26.67,16.5,0.2


In [1171]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

C:\Users\Apple\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,easy use,start dead,right away,use work,charge fast,totally dead,charge charge,product easy,hold button,easy use work,come charge,start right,dead battery,great product,say ready
Topic 1,start vehicle,simple use,fully charge,peace mind,quality product,completely dead,time charge,work advertise,come case,force start,heavy duty,start completely,easy read,start completely dead,dead battery
Topic 2,completely dead,carry case,charge phone,use charge,work perfectly,dead start,charge use,able start,charge minute,long time,say ready,wall charger,start right,use time,minute charge
Topic 3,highly recommend,great product,hold charge,charge quickly,fast charge,high quality,charge start,recommend product,definitely recommend,start immediately,start truck,turn key,come charge,right box,charge fast
Topic 4,work great,use start,use time,start right,work like,super easy,time use,use twice,like charm,work like charm,super easy use,product work,right box,easy use,totally dead


Conclusion:

Topic 0: Easy to use

Topic 1: Start the car

Topic 2: Works with completely dead battery

Topic 3: Good quality and recommended

Topic 4: Work smoothly

# Analyzing the Bad Review

In [1173]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram

#tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
#                                 min_df=0.01, stop_words='english',
#                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(2,3))

tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_lemmatization, ngram_range=(2,3))

tfidf_matrix = tfidf_model.fit_transform(Bad_Review) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

tfidf_matrix

#tfidf_matrix.toarray() #todense()

#tfidf_matrix.todense()

#Save the terms identified by TF-IDF.

C:\Users\Apple\AppData\Roaming\Python\Python38\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['make', 'u'] not in stop_words.
  warnings.warn(


In total, there are 179 reviews and 186 terms.


<179x186 sparse matrix of type '<class 'numpy.float64'>'
	with 471 stored elements in Compressed Sparse Row format>

In [1174]:
# words
tf_selected_words = tfidf_model.get_feature_names()

C:\Users\Apple\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [1175]:
# print out words
tf_selected_words[0:15]

['able issue',
 'able start',
 'air compressor',
 'attempt start',
 'attempt use',
 'bad start',
 'big engine',
 'bought month',
 'bought month ago',
 'bought product',
 'cable work',
 'car dead',
 'carry case',
 'chance use',
 'change review']

# Part 4: K-means clustering

In [1177]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [1178]:
#4.1. Analyze K-means Result

In [1179]:
# create DataFrame films from all of the input files.
product = { "Stars":hulkman_Bad["Stars"],'review': hulkman_Bad["Review Body"], 'cluster': clusters}
frame = pd.DataFrame(product, columns = ["Stars",'review', 'cluster'])

In [1180]:
frame.head(10)

,Stars,review,cluster
1,1.0,Support phone number will not accept incoming ...,2
8,3.0,I bought this product a few weeks ago not expe...,2
9,1.0,Stay away only worked 3 times and gave up,2
15,1.0,I sell cars and i use jumpers very frequently....,2
16,1.0,I watched many reviews about this Hulkman jump...,2
19,3.0,UPDATE: The seller read my review and provide...,2
21,1.0,Item did not work followed instructions and ca...,2
22,1.0,Worked great one time. Only had 32 days and lo...,2
23,1.0,car was super dead would not jump it off. han...,2
25,1.0,"Portable, yes. Effective, no.\n\nThis couldn’t...",2


In [1181]:
frame.groupby("cluster").agg("mean")

,Stars
cluster,
0,2.000000
1,1.000000
2,1.680723
3,1.000000
4,1.000000


In [1182]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
2,166
3,4
0,3
1,3
4,3


In [1183]:
km.cluster_centers_

# 239数的list -> cluster 0的中心点的tf-idf值
#-> assumption: 中心点的值可以代表这个cluster
#-> tf-idf值越大，对应的词越能代表这个document
#-> 选出了tf-idf最大的6个值对应的词来代表这个cluster

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.28766929,
        0.        , 0.        , 0.        , 0.  

In [1184]:
km.cluster_centers_.shape

(5, 186)

In [1185]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :15]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print()
  
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    #print (", ".join(cluster_reviews))
    print()

<Document clustering result by K-means>
Cluster 0 words:wo work,light come,ready light come,ready light,work like,start return,need use,great time,junk waste,jumpstart need,juice start,item work,item return,hold charge,guy come,
Cluster 0 reviews (3 reviews): 

Cluster 1 words:use twice,waste time,junk waste,error code,know start,junk work,jumpstart need,juice start,item work,item return,hold charge,guy come,great time,great easy,good deal,
Cluster 1 reviews (3 reviews): 

Cluster 2 words:waste money,work time,easy use,completely dead,start vehicle,come case,error message,time charge,error code,dead battery,return refund,stop work,start engine,force start,fully charge,
Cluster 2 reviews (166 reviews): 

Cluster 3 words:product work,work try,bought product,great time,know start,junk work,junk waste,jumpstart need,juice start,item work,item return,hold charge,guy come,great easy,lead light,
Cluster 3 reviews (4 reviews): 

Cluster 4 words:return window close,return window,window close,re

Conclusion:

Cluster 0: Do not work

Cluster 1: Waste time and error code

Cluster 2: Waste money

Cluster 3: Try many time to start the car

Cluster 4: Hard to return

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [1187]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [1188]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(179, 5)
[[0.2        0.2        0.2        0.2        0.2       ]
 [0.80172193 0.04923843 0.04988296 0.04980741 0.04934927]
 [0.0849565  0.08305745 0.08306045 0.66586899 0.08305662]
 [0.59874771 0.10002307 0.10118211 0.10002525 0.10002186]
 [0.2        0.2        0.2        0.2        0.2       ]
 [0.0496626  0.05206634 0.04895045 0.04814508 0.80117554]
 [0.76220281 0.05868591 0.0588582  0.05887929 0.0613738 ]
 [0.07327975 0.07328    0.70589425 0.07328184 0.07426417]
 [0.59895833 0.10002311 0.10002748 0.10096918 0.10002189]
 [0.2        0.2        0.2        0.2        0.2       ]
 [0.05589622 0.0574167  0.77669532 0.05438257 0.05560919]
 [0.7756352  0.05599704 0.0550843  0.05809201 0.05519146]
 [0.05101723 0.05247566 0.05006445 0.05097315 0.79546951]
 [0.08400332 0.66618998 0.08348141 0.08294925 0.08337604]
 [0.05024509 0.79614302 0.04996762 0.05344552 0.05019875]
 [0.067578   0.06672635 0.7321978  0.0667724  0.06672546]
 [0.06231167 0.06234971 0.06224001 0.75031418 0.06278442]
 [0.0

In [1189]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 186)
[[0.20003702 1.2003444  0.64025973 0.20004086 0.20004575 0.6319603
  0.20004688 0.66842955 0.66842955 0.53222175 0.63121938 0.20001368
  0.20003567 1.86470822 1.12886209 0.20004573 1.10463156 0.2000155
  0.20003885 0.53132986 0.20005486 0.20007224 0.2000176  0.20004896
  0.20006319 0.59416531 0.20005836 0.20005191 0.20004467 0.51222278
  0.20003652 2.93738073 1.22776721 1.20156461 0.20006404 0.20002386
  0.20085223 0.59203686 0.61911899 0.20005753 1.20079423 0.20004105
  0.20004657 0.20004396 0.56238077 0.20004254 0.98894293 1.24914469
  0.20002687 0.20001731 0.9045576  0.9045576  0.61863409 1.33771565
  0.56402534 0.20004582 0.66841614 0.20005026 0.20004105 0.77716505
  0.20003752 0.61811284 0.20004399 0.20005277 0.20005728 0.20027175
  0.20004429 0.4908312  0.20004667 0.20004563 0.20005102 0.20004215
  0.20003853 0.20003853 1.55446889 0.20007549 0.20005303 0.2000495
  1.49281643 0.97828282 1.01431016 0.20003261 0.20005141 0.50872272
  0.20013333 0.20027175 0.66978721 0.20005

In [1190]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(hulkman_Bad))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.20,0.20,0.20,0.20,0.20,0
Doc1,0.80,0.05,0.05,0.05,0.05,0
Doc2,0.08,0.08,0.08,0.67,0.08,3
Doc3,0.60,0.10,0.10,0.10,0.10,0
Doc4,0.20,0.20,0.20,0.20,0.20,0
Doc5,0.05,0.05,0.05,0.05,0.80,4
Doc6,0.76,0.06,0.06,0.06,0.06,0
Doc7,0.07,0.07,0.71,0.07,0.07,2
Doc8,0.60,0.10,0.10,0.10,0.10,0
Doc9,0.20,0.20,0.20,0.20,0.20,0


In [1191]:
df_document_topic['topic'].value_counts().to_frame()

,topic
0,73
1,31
4,31
3,24
2,20


In [1192]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.round(2).head()

[[0.20003702 1.2003444  0.64025973 0.20004086 0.20004575 0.6319603
  0.20004688 0.66842955 0.66842955 0.53222175 0.63121938 0.20001368
  0.20003567 1.86470822 1.12886209 0.20004573 1.10463156 0.2000155
  0.20003885 0.53132986 0.20005486 0.20007224 0.2000176  0.20004896
  0.20006319 0.59416531 0.20005836 0.20005191 0.20004467 0.51222278
  0.20003652 2.93738073 1.22776721 1.20156461 0.20006404 0.20002386
  0.20085223 0.59203686 0.61911899 0.20005753 1.20079423 0.20004105
  0.20004657 0.20004396 0.56238077 0.20004254 0.98894293 1.24914469
  0.20002687 0.20001731 0.9045576  0.9045576  0.61863409 1.33771565
  0.56402534 0.20004582 0.66841614 0.20005026 0.20004105 0.77716505
  0.20003752 0.61811284 0.20004399 0.20005277 0.20005728 0.20027175
  0.20004429 0.4908312  0.20004667 0.20004563 0.20005102 0.20004215
  0.20003853 0.20003853 1.55446889 0.20007549 0.20005303 0.2000495
  1.49281643 0.97828282 1.01431016 0.20003261 0.20005141 0.50872272
  0.20013333 0.20027175 0.66978721 0.20005303 0.200

C:\Users\Apple\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,able issue,able start,air compressor,attempt start,attempt use,bad start,big engine,bought month,bought month ago,bought product,...,work month,work really,work time,work tried,work try,work waste,work waste money,write instruction,year old,year wo
Topic0,0.20,1.20,0.64,0.20,0.20,0.63,0.20,0.67,0.67,0.53,...,0.20,0.20,1.75,0.93,0.20,0.20,0.20,0.20,1.02,0.2
Topic1,0.20,0.20,0.82,0.20,0.71,0.20,0.20,0.72,0.72,0.20,...,0.20,1.41,0.20,0.20,0.79,0.20,0.20,0.46,0.20,0.2
Topic2,0.56,0.59,0.20,0.20,0.20,0.59,0.45,0.20,0.20,0.20,...,0.53,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.20,0.8
Topic3,0.66,0.20,0.20,0.97,0.67,0.20,0.20,0.20,0.20,0.94,...,0.20,0.20,1.98,0.20,0.94,0.20,0.20,0.81,0.20,0.2
Topic4,0.20,0.20,0.20,0.63,0.20,0.20,0.53,0.20,0.20,0.20,...,0.62,0.20,0.20,0.54,0.20,1.42,1.42,0.20,0.23,1.2


In [1193]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

C:\Users\Apple\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,come case,product work,chance use,work time,like work,maybe start,force start,review receive,error message,completely dead,receive tried,wo charge,connect cable,use twice,dead car
Topic 1,easy use,dead battery,run air,fast charge,use charge,really want,say ready,time charge,tow truck,quickly use,work really,couple time,error code,use time,charge phone
Topic 2,unable start,try turn,start engine,crank engine,start car,work great,charge past,start second,charge plug,engine start,customer service,unfortunately stop,jumpstart need,recommend product,year wo
Topic 3,start vehicle,carry case,work time,waste money,fully charge,product work,work good,wait min,vehicle start,try start,long charge,charge hour,error message,attempt start,junk waste
Topic 4,return refund,waste money,piece junk,cold weather,use product,unable return,window close,return window,return window close,work waste money,work waste,code dead,try work,charge truck,cable work


Conclusion:

Topic 0: Do not work

Topic 1: Hard to use

Topic 2: Can not start the car

Topic 3: No carry Case

Topic 4: Need to return